In [2]:
import numpy as np
import pandas as pd
import textwrap as tw
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
tweets = pd.read_csv("../tweets/alltweets.csv")
bitcoin_tweets = tweets[tweets["tweet"].str.match(r".*([b,B]itcoin|BTC).*")].reset_index()
bitcoin_data = pd.read_csv("../bitcoin/bitcoin.csv")

In [20]:
def plot_asset_and_tweets(asset_df, tweet_df):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=asset_df["Date"], 
                             y=asset_df["High"], 
                             name="Asset Price"))

    asset_df["slope"] = np.gradient(asset_df.High)
    tweet_df["created_at"] = pd.to_datetime(tweet_df["created_at"]).map(lambda x: x.replace(hour=0, minute=0, second=0))
    asset_df["Date"] = pd.to_datetime(asset_df["Date"]).map(lambda x: x.replace(hour=0, minute=0, second=0))
    merged = pd.merge(left=tweet_df, left_on="created_at", right=asset_df, right_on="Date")

    fig.add_trace(go.Scatter(x=merged["created_at"], 
                             y=merged["High"], 
                             name="Relevent Tweets", 
                             mode='markers',
                             hovertemplate ="<br>".join([
                                            'Price: %{y:.2f}',
                                            'Price slope: %{customdata}',
                                            'Date: %{x}',
                                            'Tweet: %{text}',
                                            '<extra></extra>',
                                            ]),
                             marker_symbol="x",
                             text=merged["tweet"].map(lambda s: "<br>".join(tw.wrap(s))),
                             customdata=["slope"]))
    return fig

plot_asset_and_tweets(bitcoin_data, bitcoin_tweets).show()